# A Study of Feature Importance in the Forest Cover Type Prediction Dataset


Data source: https://www.kaggle.com/c/forest-cover-type-prediction

In [1]:
import os
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from pandas.tools.plotting import scatter_matrix

from IPython.core.display import display, HTML
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif 

%matplotlib inline

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load Data

In [2]:
forest = pd.read_csv("data/train.csv", index_col=0) 
forest.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


## Create Functions for Feature Engineering

In [3]:
def labelSoilType(row):
    """
    Label soil types
    """
    for i in range(len(row)):
        if row[i] == 1:
            return 'Soil_Type'+str(i)
        
def azimuth_to_abs(x):
    """
    Only care about the absolute angle from 0 w/o respect to direction
    """
    if x>180:
        return 360-x
    else:
        return x

## Feature Engineering
#### Much of the inspiration of these can be found in the feature exploration notebooks

In [4]:
# Create Soil Type Buckets
soil_types = pd.read_csv('soil_types.csv').set_index('Soil Type')
forest['Soil Type'] = forest[['Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7',
       'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15',
       'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
       'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
       'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
       'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
       'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
       'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40']].apply(lambda row: labelSoilType(row), axis=1)
forest = pd.merge(forest, soil_types, how='left', left_on='Soil Type', right_index=True)
del forest['Soil Type'] # Delete string column

# Create feature to that transforms azimuth to its absolute value
forest['Aspect2'] = forest.Aspect.map(azimuth_to_abs)
forest['Aspect2'].astype(int)

# Create feature that determines if the patch is above sea level
forest['Above_Sealevel'] = (forest.Vertical_Distance_To_Hydrology>0).astype(int)

# Bin the Elevation Feature: check the feature exploration notebook for motivation
bins = [0, 2600, 3100, 8000]
group_names = [1, 2, 3]
forest['Elevation_Bucket'] = pd.cut(forest['Elevation'], bins, labels=group_names)
forest['Elevation_0_2600'] = np.where(forest['Elevation_Bucket']== 1, 1, 0)
forest['Elevation_2600_3100'] = np.where(forest['Elevation_Bucket']== 2, 1, 0)
forest['Elevation_3100_8000'] = np.where(forest['Elevation_Bucket']== 3, 1, 0)
forest['Elevation_0_2600'].astype(int)
forest['Elevation_2600_3100'].astype(int)
forest['Elevation_3100_8000'].astype(int)
del forest['Elevation_Bucket']

# Create a feature for no hillshade at 3pm
forest['3PM_0_Hillshade'] = (forest.Hillshade_3pm == 0).astype(int)

#Direct distance to hydrology
forest['Direct_Distance_To_Hydrology'] = np.sqrt((forest.Vertical_Distance_To_Hydrology**2) + \
    (forest.Horizontal_Distance_To_Hydrology**2)).astype(float).round(2)


soil_types= ['Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6',
       'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
       'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
       'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
       'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
       'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
       'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40', 'Cover_Type']

column_list = forest.columns.tolist()
column_list = [c for c in column_list if c[:9] != 'Soil_Type']
column_list.insert(10, 'Direct_Distance_To_Hydrology')
column_list.insert(11, 'Elevation_0_2600')
column_list.insert(12, 'Elevation_2600_3100')
column_list.insert(13, 'Elevation_3100_8000')
column_list.insert(14, 'Aspect2')
column_list.insert(15, 'Above_Sealevel')
column_list.insert(16, '3PM_0_Hillshade')
column_list.extend(soil_types)
columns = []
for col in column_list:
    if col not in columns:
        if col != 'Cover_Type':
            columns.append(col)
columns.append('Cover_Type')
        

forest = forest[columns]
forest.fillna(0,inplace=True) # Replace nans with 0 for our soil type bins
forest.shape

(15120, 99)

In [5]:
forest.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Direct_Distance_To_Hydrology,Elevation_0_2600,Elevation_2600_3100,Elevation_3100_8000,Aspect2,Above_Sealevel,3PM_0_Hillshade,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Cryumbrepts,Bross,Cryorthents,Legault,Granile,Bouldery,Till substratum,Como,Leighcan,Moran,Rock,Outcrop,Stony,Very Stony,Extremely Stony,Rubbly,Warm,Complex,Cathedral,Vanet,Ratake,Haploborolis,Wetmore,Gothic,Limber,Troutville,Bullwark,Catamount,Legault.1,Land,Pachic Argiborolis,Aquolis,Cryaquolis,Cryoborolis,Gateview,Rogert,Typic,Cryaquepts,Borohemists,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,258.00,1,0,0,51,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,212.08,1,0,0,56,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,275.77,0,1,0,139,1,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,269.24,0,1,0,155,1,0,1,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,153.00,1,0,0,45,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


## Remove Base Features with no Modeling Value

In [6]:
to_remove = [] # features to drop
for c in forest.columns.tolist():
    if forest[c].std() == 0:
        to_remove.append(c)
forest = forest.drop(to_remove, 1)
print("Dropped the following columns: \n")
for r in to_remove:
    print (r)

Dropped the following columns: 

Wetmore
Pachic Argiborolis
Aquolis


In [7]:
forest.shape

(15120, 96)

## Add Feature Interactions

In [8]:
for i in range(forest.shape[1]-1):
    for j in range(54):
        if i != j:
            forest[forest.columns.tolist()[i]+"_"+forest.columns.tolist()[j]] = forest[forest.columns.tolist()[i]]*forest[forest.columns.tolist()[j]]

In [9]:
forest.shape

(15120, 5172)

## Remove Columns That Have No Value

In [10]:
to_remove = [] # features to drop
for c in forest.columns.tolist():
    if forest[c].std() == 0:
        to_remove.append(c)
forest = forest.drop(to_remove, 1)
print("Dropped the following columns: \n")
for r in to_remove:
    print (r)

Dropped the following columns: 

Hillshade_3pm_3PM_0_Hillshade
Elevation_0_2600_Elevation_2600_3100
Elevation_0_2600_Elevation_3100_8000
Elevation_0_2600_Wilderness_Area2
Elevation_0_2600_Cryumbrepts 
Elevation_0_2600_Bross
Elevation_0_2600_Cryorthents
Elevation_0_2600_Granile
Elevation_0_2600_Moran
Elevation_0_2600_Gothic
Elevation_0_2600_Rogert
Elevation_2600_3100_Elevation_0_2600
Elevation_2600_3100_Elevation_3100_8000
Elevation_2600_3100_Cryumbrepts 
Elevation_2600_3100_Bross
Elevation_2600_3100_Cryorthents
Elevation_3100_8000_Elevation_0_2600
Elevation_3100_8000_Elevation_2600_3100
Elevation_3100_8000_Wilderness_Area4
Elevation_3100_8000_Rubbly
Elevation_3100_8000_Warm
Elevation_3100_8000_Cathedral
Elevation_3100_8000_Vanet
Elevation_3100_8000_Ratake
Elevation_3100_8000_Haploborolis
Elevation_3100_8000_Gothic
Elevation_3100_8000_Limber
Elevation_3100_8000_Troutville
Elevation_3100_8000_Bullwark
Elevation_3100_8000_Cryoborolis
Elevation_3100_8000_Gateview
Above_Sealevel_Gothic
3PM_

Soil_Type14_Gothic
Soil_Type14_Limber
Soil_Type14_Troutville
Soil_Type14_Legault.1
Soil_Type14_Cryaquolis
Soil_Type14_Cryoborolis
Soil_Type14_Gateview
Soil_Type14_Rogert
Soil_Type16_Elevation_3100_8000
Soil_Type16_3PM_0_Hillshade
Soil_Type16_Wilderness_Area2
Soil_Type16_Cryumbrepts 
Soil_Type16_Bross
Soil_Type16_Cryorthents
Soil_Type16_Legault
Soil_Type16_Granile
Soil_Type16_Bouldery
Soil_Type16_Till substratum
Soil_Type16_Como
Soil_Type16_Leighcan
Soil_Type16_Moran
Soil_Type16_Rock
Soil_Type16_Outcrop
Soil_Type16_Stony
Soil_Type16_Very Stony
Soil_Type16_Extremely Stony
Soil_Type16_Rubbly
Soil_Type16_Warm
Soil_Type16_Complex
Soil_Type16_Cathedral
Soil_Type16_Vanet
Soil_Type16_Ratake
Soil_Type16_Haploborolis
Soil_Type16_Gothic
Soil_Type16_Limber
Soil_Type16_Troutville
Soil_Type16_Bullwark
Soil_Type16_Catamount
Soil_Type16_Legault.1
Soil_Type16_Land
Soil_Type16_Gateview
Soil_Type16_Rogert
Soil_Type17_Elevation_3100_8000
Soil_Type17_Wilderness_Area1
Soil_Type17_Wilderness_Area2
Soil_Type1

In [11]:
forest.shape

(15120, 2803)

In [12]:
forest.head()

Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
Id                                                               
1        2596      51      3                               258   
2        2590      56      2                               212   
3        2804     139      9                               268   
4        2785     155     18                               242   
5        2595      45      2                               153   

    Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
Id                                                                    
1                                0                              510   
2                               -6                              390   
3                               65                             3180   
4                              118                             3090   
5                               -1                              391   

    Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
Id                                                 
1             221             232            148   
2             220             235            151   
3             234             238            135   
4             238             238            122   
5             220             234            150   

    Horizontal_Distance_To_Fire_Points  Direct_Distance_To_Hydrology  \
Id                                                                     
1                                 6279                        258.00   
2                                 6225                        212.08   
3                                 6121                        275.77   
4                                 6211                        269.24   
5                                 6172                        153.00   

    Elevation_0_2600  Elevation_2600_3100  Elevation_3100_8000  Aspect2  \
Id                                                                        
1                  1                    0                    0       51   
2                  1                    0                    0       56   
3                  0                    1                    0      139   
4                  0                    1                    0      155   
5                  1                    0                    0       45   

    Above_Sealevel  3PM_0_Hillshade  Wilderness_Area1  Wilderness_Area2  \
Id                                                                        
1                0                0                 1                 0   
2                0                0                 1                 0   
3                1                0                 1                 0   
4                1                0                 1                 0   
5                0                0                 1                 0   

    Wilderness_Area3  Wilderness_Area4  Cryumbrepts   Bross  Cryorthents  \
Id                                                                         
1                  0                 0           0.0    0.0          0.0   
2                  0                 0           0.0    0.0          0.0   
3                  0                 0           0.0    0.0          0.0   
4                  0                 0           0.0    0.0          0.0   
5                  0                 0           0.0    0.0          0.0   

    Legault  Granile  Bouldery  Till substratum  Como  Leighcan  Moran  Rock  \
Id                                                                             
1       0.0      0.0       0.0              0.0   0.0       1.0    0.0   1.0   
2       0.0      0.0       0.0              0.0   0.0       1.0    0.0   1.0   
3       0.0      0.0       0.0              0.0   0.0       0.0    0.0   0.0   
4       1.0      0.0       0.0              0.0   1.0       0.0    0.0   0.0   
5       0.0      0.0       0.0              0.0   0.0       1.0    0.0   1.0   

    Outcrop  Stony  Very St


### Transform the continuous features
###### We will try Normalization, Standardized Scaling, and MinMax Scaling
###### Note: there is no need to impute any data points as this is a pretty clean data set

In [13]:
# Initialize lists and variables
chunk_size = 0.1 #Validation chunk size
seed = 0 # Use the same random seed to ensure consistent validation chunk usage

ranks = [] #array of importance rank of all features
X_all = [] # all features
X_all_add = [] # Additionally we will make a list of subsets
rem = [] # columns to be dropped
i_rem = [] # indexes of columns to be dropped
trans_list = [] # Transformations
comb = [] # combinations
comb.append("All+1.0")

ratio_list = [0.75,0.50,0.25] #Select top 75%, 50%, 25% of features
features = [] # feature selection models
model_features = [] # names of feature selection models

# reorder the data to have continuous variables come first
continuous = [] # continuous variables
categorical = [] # categorical variables
final_columns = [] # final columns list
for col in forest.columns.tolist():
    if col in to_remove:
        pass
    elif col == 'Cover_Type':
        pass
    elif forest[col].nunique() > 4:
        continuous.append(col)
    else:
        categorical.append(col)

final_columns.extend(continuous)
final_columns.extend(categorical)
final_columns.append('Cover_Type')
forest = forest[final_columns]
num_rows, num_cols = forest.shape
cols = forest.columns
size = len(continuous) # Number of continuous columns

i_cols = []
for i in range(0,num_cols-1):
    i_cols.append(i)

# Create the data arrays for model building
val_array = forest.values
X = val_array[:,0:(num_cols-1)]
y = val_array[:,(num_cols-1)]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=chunk_size, random_state=seed)
X_all.append(['Orig','All', X_train,X_val,1.0,cols[:num_cols-1],rem,ranks,i_cols,i_rem])

In [14]:
# Standardize the data

X_temp = StandardScaler().fit_transform(X_train[:,0:size])
X_val_temp = StandardScaler().fit_transform(X_val[:,0:size])

# Recombine data
X_con = np.concatenate((X_temp,X_train[:,size:]),axis=1)
X_val_con = np.concatenate((X_val_temp,X_val[:,size:]),axis=1)

X_all.append(['StdSca','All', X_con,X_val_con,1.0,cols,rem,ranks,i_cols,i_rem])

In [15]:
# MinMax Scale the data

X_temp = MinMaxScaler().fit_transform(X_train[:,0:size])
X_val_temp = MinMaxScaler().fit_transform(X_val[:,0:size])

# Recombine data
X_con = np.concatenate((X_temp,X_train[:,size:]),axis=1)
X_val_con = np.concatenate((X_val_temp,X_val[:,size:]),axis=1)

X_all.append(['MinMax', 'All', X_con,X_val_con,1.0,cols,rem,ranks,i_cols,i_rem])

In [16]:
#Normalize the data

X_temp = Normalizer().fit_transform(X_train[:,0:size])
X_val_temp = Normalizer().fit_transform(X_val[:,0:size])

# Recombine data
X_con = np.concatenate((X_temp,X_train[:,size:]),axis=1)
X_val_con = np.concatenate((X_val_temp,X_val[:,size:]),axis=1)

X_all.append(['Norm', 'All', X_con,X_val_con,1.0,cols,rem,ranks,i_cols,i_rem])

In [17]:
# Add transformation to the list
for trans,name,X,X_val,v,cols_list,rem_list,rank_list,i_cols_list,i_rem_list in X_all:
    trans_list.append(trans)
trans_list

['Orig', 'StdSca', 'MinMax', 'Norm']

### Create the classifiers for measuring importance
- Extra Trees Classifier
- Gradient Boosting Classifier
- Random Forest Classifier
- XGBoost Classifier
- Random Feature Elimination Classifier
- Select Percentile

In [20]:
# Add Extra Trees Classifier
n = 'Extra Trees Classifier'
model_features.append(n)
for val in ratio_list:
    comb.append("%s+%s" % (n,val))
    features.append([n,val,ExtraTreesClassifier(n_estimators=num_cols-1,max_features=val,n_jobs=-1,random_state=seed)])      

# Add Gradient Boosting Classifier
n = 'Gradient Boosting Classifier'
model_features.append(n)
for val in ratio_list:
    comb.append("%s+%s" % (n,val))
    features.append([n,val,GradientBoostingClassifier(n_estimators=num_cols-1,max_features=val,random_state=seed)])   

# Add Random Forest Classifiers
n = 'Random Forest Classifier'
model_features.append(n)
for val in ratio_list:
    comb.append("%s+%s" % (n,val))
    features.append([n,val,RandomForestClassifier(n_estimators=num_cols-1,max_features=val,n_jobs=-1,random_state=seed)])   

# Add XGBoost Classifier
n = 'XGBoost Classifier'
model_features.append(n)
for val in ratio_list:
    comb.append("%s+%s" % (n,val))
    features.append([n,val,XGBClassifier(n_estimators=num_cols-1,seed=seed)])   

# Add RFE
model = LogisticRegression(random_state=seed,n_jobs=-1)
n = 'Random Feature Elimination'
model_features.append(n)
for val in ratio_list:
    comb.append("%s+%s" % (n,val))
    features.append([n,val,RFE(model,val*(num_cols-1))])
    
# Add Select Percentile
n = 'Select Percentile'
model_features.append(n)
for val in ratio_list:
    comb.append("%s+%s" % (n,val))
    features.append([n,val,SelectPercentile(score_func=f_classif,percentile=val*100)])   


### Run All the Models

In [ ]:
# Determine feature importance for each model and transformation combination

for trans, s, X, X_val, d, cols, rem, ra, i_cols, i_rem in X_all:
    for name, v, model in features:
        print ("Training ", name, "with", v*100, "% features and a", trans, "transformation")
        # Train the model against y
        model.fit(X,y_train)

        # Combine importance and index of the column in the array joined
        joined = []
        if name == "Random Feature Elimination":
            for i, pred in enumerate(list(model.ranking_)):
                joined.append([i,cols[i],pred])
        elif name == "Select Percentile":
            for i, pred in enumerate(list(model.scores_)):
                joined.append([i,cols[i],pred])
        else:
            for i, pred in enumerate(list(model.feature_importances_)):
                joined.append([i,cols[i],pred])

        cols_list = [] # List of names of columns selected
        i_cols_list = [] # Indexes of columns selected
        rank_list =[] # Ranking of all the columns
        rem_list = [] # List of columns not selected
        i_rem_list = [] # Indexes of columns not selected

        joined_sorted = sorted(joined, key=lambda x: -x[2]) # Sort in descending order  
        rem_start = int((v*(num_cols-1))) # Starting point of the columns to be dropped

        # Split the array. Store selected columns in cols_list and removed in rem_list
        for j, (i, col, x) in enumerate(list(joined_sorted)):
            rank_list.append([i,j])
            if(j < rem_start):
                cols_list.append(col)
                i_cols_list.append(i)   
            else:
                rem_list.append(col)
                i_rem_list.append(i)

        # Sort the rank_list and store only the ranks. Drop the index 
        # Append model name, array, columns selected and columns to be removed to the additional list        
        X_all_add.append([trans,name,X,X_val,v,cols_list,rem_list,[x[1] for x in sorted(rank_list,key=lambda x:x[0])],i_cols_list,i_rem_list])    

Training  Extra Trees Classifier with 75.0 % features and a Orig transformation


### Plot the results

In [ ]:
#Set figure size
plt.rc("figure", figsize=(30, 10))

#Plot a graph for different feature selectors        
for f_name in model_features:
    #Array to store the list of combinations
    leg=[]
    fig, ax = plt.subplots()
    #Plot each combination
    for trans,name,X,X_val,v,cols_list,rem_list,rank_list,i_cols_list,i_rem_list in X_all_add:
        if(name==f_name):
            plt.plot(rank_list)
            leg.append(trans+"+"+name+"+%s"% v)
            plt.title(name,size=20)
    #Set the tick names to names of columns
    ax.set_xticks(range(num_cols-1))
    ax.set_xticklabels(cols[:num_cols-1],rotation='vertical')
    #Display the plot
    plt.legend(leg,loc='best')    
    #Plot the rankings of all the features for all combinations
    plt.show()

## Select the Important Features

In [ ]:
rank_df = pd.DataFrame(data=[x[7] for x in X_all_add],columns=cols[:num_cols-1])
rank_median = rank_df.median()
rank_median.sort()
rank_df = rank_df[rank_median.index.values.tolist()]

In [ ]:
fig = plt.figure(1,figsize=(40,15))
plt.rcParams['ytick.labelsize'] = 24
ax = fig.add_subplot(111)
bp = ax.boxplot(rank_df.values)
ax.set_xticklabels(rank_df.columns.tolist(), rotation=40, ha='right', size='xx-large')
for box in bp['boxes']:
    box.set(color='blue', linewidth=2.5)
for whisker in bp['whiskers']:
    whisker.set(color='blue', linewidth=2.5)
for median in bp['medians']:
    median.set(color='red', linewidth=2.5)

plt.savefig('visualizations/median_feature_importance_interactions.png')    
plt.show()

In [ ]:
rank_median